In [41]:
input_file_path = "prepare_data/dev_input.csv"
model_name = "llama3.2"
dialect = "sqlite"
output_file_path = "inference_data/dev_inference.txt"

In [42]:
from langchain_core.prompts import PromptTemplate

common_prompt_template = PromptTemplate.from_template(
    """You are an expert in {dialect}. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of {dialect} to generate an SQL statement that answers the [User Question].

[User Question]
{user_question}

[Database Schema]
{schema}

[Reference Information]
None

ONLY OUTPUT THE SQL STATEMENT, NO OTHER TEXT.
"""
)

In [43]:
model_config = {
    "llama3.2": {
        "model_name": "llama3.2",
        "model_config_init": {
            "temperature": 0.0,
        },
        "prompt_template": common_prompt_template,
    }
}

In [44]:
from langchain_ollama import ChatOllama


llm = ChatOllama(
    model=model_config[model_name]["model_name"],
    **model_config[model_name]["model_config_init"]
)
llm.invoke("Hello world")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-20T08:32:28.93341Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6579471880, 'load_duration': 4148442506, 'prompt_eval_count': 27, 'prompt_eval_duration': 922256915, 'eval_count': 25, 'eval_duration': 1507261224, 'model_name': 'llama3.2'}, id='run--c6a97822-810a-43f7-882e-683b8db247ba-0', usage_metadata={'input_tokens': 27, 'output_tokens': 25, 'total_tokens': 52})

In [45]:
prompt_template = model_config[model_name]["prompt_template"]


def generate_sql(user_question: str, schema_list: list[str], dialect: str):
    message = prompt_template.invoke(
        {
            "user_question": user_question,
            "schema": "\n".join(schema_list),
            "dialect": dialect,
        }
    )
    message = llm.invoke(message).content
    # format the message to be a valid sql statement
    message = (
        message.strip()
        .replace("```sql", "")
        .replace("```", "")
        .replace("\n", "")
        .replace("\t", "")
    )
    print(f"Generated answer for question: {user_question[:20]}...")
    return message

In [46]:
generate_sql(
    "How many singers do we have?",
    [
        'CREATE TABLE "singer" (\n"Singer_ID" int,\n"Name" text,\n"Country" text,\n"Song_Name" text,\n"Song_release_year" text,\n"Age" int,\n"Is_male" bool,\nPRIMARY KEY ("Singer_ID")\n)'
    ],
    dialect,
)

Generated answer for question: How many singers do ...


'SELECT COUNT(*) FROM singer;'

In [47]:
import pandas as pd

df = pd.read_csv(input_file_path)
df

,question_number,question,db_id,tables,schemas
0,0,How many singers do we have?,concert_singer,['singer'],"['CREATE TABLE ""singer"" (\n""Singer_ID"" int,\n""..."
1,1,What is the total number of singers?,concert_singer,['singer'],"['CREATE TABLE ""singer"" (\n""Singer_ID"" int,\n""..."
2,2,"Show name, country, age for all singers ordere...",concert_singer,['singer'],"['CREATE TABLE ""singer"" (\n""Singer_ID"" int,\n""..."
3,3,"What are the names, countries, and ages for ev...",concert_singer,['singer'],"['CREATE TABLE ""singer"" (\n""Singer_ID"" int,\n""..."
4,4,"What is the average, minimum, and maximum age ...",concert_singer,['singer'],"['CREATE TABLE ""singer"" (\n""Singer_ID"" int,\n""..."
...,...,...,...,...,...
1029,1029,What are the citizenships that are shared by s...,singer,['singer'],"['CREATE TABLE ""singer"" (\n""Singer_ID"" int,\n""..."
1030,1030,How many available features are there in total?,real_estate_properties,['Other_Available_Features'],['CREATE TABLE `Other_Available_Features` (\n`...
1031,1031,What is the feature type name of feature AirCon?,real_estate_properties,"['Other_Available_Features', 'Ref_Feature_Types']",['CREATE TABLE `Other_Available_Features` (\n`...
1032,1032,Show the property type descriptions of propert...,real_estate_properties,"['Properties', 'Ref_Property_Types']",['CREATE TABLE `Properties` (\n`property_id` I...


In [48]:
inference_df = df.assign(
    answer=lambda df_: df_.apply(
        lambda row: generate_sql(row.question, row.schemas, dialect), axis=1
    )
)
inference_df

Generated answer for question: What are the names o...
Generated answer for question: Which are the car ma...
Generated answer for question: What are the ids and...
Generated answer for question: What are the id and ...
Generated answer for question: What are the ids and...
Generated answer for question: Which country does A...
Generated answer for question: What country is Jetb...
Generated answer for question: What is the abbrevia...
Generated answer for question: Which abbreviation c...
Generated answer for question: List all airline nam...
Generated answer for question: What are the airline...
Generated answer for question: List the airport cod...
Generated answer for question: Give the airport cod...


KeyboardInterrupt: 

In [38]:
with open(output_file_path, "w+") as f:
    answer_list = inference_df["answer"].tolist()
    f.write("\n".join(answer_list))